In [1]:
"""
Creates reference spectra of the 3 soft bgd components, used by
nuskybgd_image, to produce counts images for arbitrary energy bands.  Only
needs to be run once unless the nuabs parameters change in the auxil
directory.

Required files:

RMF files that include detector absorption (see absrmf.py).

Output:

Simulated spectra using Xspec's fakeit, plus an Xspec save file (.xcm) that
can be used to remake them in Xspec.
"""
import xspec

In [15]:
# CXB params, from Boldt '87
cxb_pl = 1.29
cxb_norm = 0.002353
cxb_ecut = 41.13

# Simulated exposure time (assumed elsewhere, do not change!)
texp = 1.0e9

################## Not in use #######################
# Soft neutron component photon index
neutpl = 4.8

# GRXE params
glenergy = 6.7
wdmass = 0.6
#####################################################

In [3]:
import os

In [4]:
os.chdir('/Users/qw/astro/nustar/IC342_X1/90201039002/event_cl/bgd')

In [21]:
auxil = '/Users/qw/astro/nustar/nuskybgd-idl/auxil'

In [8]:
ls det??.rmf

det0A.rmf  det1A.rmf  det2A.rmf  det3A.rmf
det0B.rmf  det1B.rmf  det2B.rmf  det3B.rmf


In [12]:
m = xspec.Model('cutoffpl')

In [16]:
m.cutoffpl.PhoIndex = cxb_pl
m.cutoffpl.HighECut = cxb_ecut
m.cutoffpl.norm = cxb_norm

In [17]:
xspec.AllModels.show()

In [36]:
for idet in range(4):
    for ab in ('A', 'B'):
        # If there is existing spectrum loaded, RMF and ARF in FakeSettings 
        # will be ignored (the 'from existing spectra' flow). We need to do
        # the 'from scratch' flow.
        xspec.AllData.clear()
        
        fakesettings_ap = xspec.FakeitSettings(
            response='det%d%s.rmf' % (idet, ab),
            arf='%s/be.arf' % auxil,
            exposure=texp,
            correction=0.0,
            fileName='aperbgd%s_det%d.pha' % (ab, idet)
        )
        fakesettings_fcxb = xspec.FakeitSettings(
            response='det%d%s.rmf' % (idet, ab),
            arf='%s/fcxb%s.arf' % (auxil, ab),
            exposure=texp,
            correction=0.0,
            fileName='fcxbbgd%s_det%d.pha' % (ab, idet)
        )
        xspec.AllData.fakeit(
        nSpectra=2,
        settings=[fakesettings_ap, fakesettings_fcxb],
        applyStats=False,
        filePrefix='')